In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume-dataset/resume_data.csv


In [54]:
df = pd.read_csv("/kaggle/input/resume-dataset/resume_data.csv");
df.head()

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,[None],[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000


In [55]:
import pandas as pd
import re

def clean_major_fields(df, column_name='major_field_of_studies'):
    df_cleaned = df.copy()

    abbreviations = {
        "cse": "computer science",
        "cs": "computer science",
        "it": "information technology",
        "computer": "computer science",
        "computers": "computer science",
        "ai": "artificial intelligence",
        "ml": "machine learning",
        "ds": "data science",
        "ece": "electronics engineering",
        "eee": "electrical engineering",
        "ee": "electrical engineering",
        "electrical": "electrical engineering",
        "electronics": "electronics engineering",
        "me": "mechanical engineering",
        "ce": "civil engineering",
        "che": "chemical engineering",
        "mechanical": "mechanical engineering",
        "civil": "civil engineering",
        "chemical": "chemical engineering",
        "finance": "finance",
        "accounting": "accounting",
        "business": "business administration",
        "management": "business administration",
        "marketing": "marketing",
        "statistics": "statistics",
        "economics": "economics",
        "biology": "biology",
        "chemistry": "chemistry",
        "physics": "physics",
        "math": "maths",
        "mathematics": "maths"
    }

    split_pattern = re.compile(r"[\/,&;|\s]+")

    empty_values = {"n/a", "none", "na", "null", "", "nan", "n, a", "n,a", "n", "a"}

    def process_value(value):
        if pd.isna(value) or value is None:
            return []
        
        value_str = str(value).lower().strip()

        if value_str in empty_values:
            return []

        value_str = value_str.strip("[]\"'")

        items = [item.strip() for item in split_pattern.split(value_str) if item.strip()]

        cleaned_items = []
        for item in items:
            if item in empty_values:
                continue
            cleaned_item = abbreviations.get(item, item)
            if cleaned_item:
                cleaned_items.append(cleaned_item)

        seen = set()
        unique_items = [x for x in cleaned_items if not (x in seen or seen.add(x))]

        return unique_items if unique_items else []

    df_cleaned[column_name] = df_cleaned[column_name].apply(process_value)
    
    return df_cleaned


In [56]:
df = clean_major_fields(df, column_name='major_field_of_studies')

In [57]:
import pandas as pd
import re

def clean_experience_min_only(df, column_name='experience', default_value=0):
    """
    Cleans experience column to extract the minimum number of years mentioned.
    Examples:
        'At least 3 years' → 3
        '3 to 5 years' → 3
        '1 to 2 years' → 1
        NaN or invalid → default_value (e.g., 0)

    Args:
        df (pd.DataFrame): Input DataFrame
        column_name (str): The column name to clean
        default_value (int): Value to assign to missing/invalid entries

    Returns:
        pd.DataFrame: DataFrame with cleaned experience column
    """
    def extract_min_years(value):
        if pd.isna(value):
            return default_value
        # Find all numbers in the string
        numbers = re.findall(r'\d+', str(value))
        if numbers:
            return int(numbers[0])  # take the minimum
        return default_value

    df_cleaned = df.copy()
    df_cleaned[column_name] = df_cleaned[column_name].apply(extract_min_years)
    return df_cleaned


In [58]:
df = clean_experience_min_only(df, column_name='experiencere_requirement')


In [36]:
df

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],[electronics engineering],['Coca-COla'],...,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,1,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","[mathematics', 'statistics]",['BIB Consultancy'],...,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,5,NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],"[electronics engineering, telecommunication]",['Axis Bank Limited'],...,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),3,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],"[computer science, applications]","['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],[accounting],"['Company Name', 'Company Name', 'Company Name...",...,[None],[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,4,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,NaN,NaN,"['Mathematical modelling', 'Machine Learning',...",['Sanghvi College of Engineering'],['B.Tech'],['2019'],['N/A'],['N/A'],[],['BPM Foundation'],...,NaN,NaN,NaN,Data Engineer,Bachelor of Science (BSc),5,NaN,Data Platform Design\nData Pipeline Developmen...,Azure\nBig Data\nData Analytics\nETL Tools\nPo...,0.683333
9540,NaN,Expertise EDA modeler. I like to learn what my...,"['Data Analysis', 'Business Analysis', 'Machin...","['KVoCT, Pune', 'KVoCT, Pune']","['B.CA', 'M.CA']","['2018', '2020']","[None, None]","[None, None]",[],['Passionate Solution'],...,NaN,NaN,NaN,Executive/ Sr. Executive -IT,Bachelor of Science (BSc) in Computer Science ...,3,Age at most 40 years,Hardware & Software Installation\nSystem Monit...,NaN,0.650000
9541,NaN,Looking for roles related to application devel...,"['Business Analyst', 'Data Analytics', 'Data C...",['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],[],['ZigSAW'],...,NaN,NaN,NaN,Executive - VAT,BBA in Accounting and Finance,1,NaN,Mushak Forms Maintenance\nVAT Software & MS Of...,VAT and Tax,0.650000
9542,NaN,NaN,"['Machine Learning', 'Natural Language Process...","['Rajiv Gandhi Memorial University, Delhi']",['B.TECH'],['2020'],['N/A'],['N/A'],[electrical engineering],['Zynta La

In [ ]:
df

In [59]:
import re
import pandas as pd

def clean_degree_name(text):
    if pd.isna(text):
        return ''
    
    text = text.lower()

    # Abbreviation expansion
    replacements = {
    r'\bb[\s\.-]*tech\b': 'bachelor of technology',
    r'\bm[\s\.-]*tech\b': 'master of technology',
    r'\bb[\s\.-]*sc\b': 'bachelor of science',
    r'\bm[\s\.-]*sc\b': 'master of science',
    r'\bb[\s\.-]*e\b': 'bachelor of engineering',
    r'\bm[\s\.-]*e\b': 'master of engineering',
    r'\bb[\s\.-]*a\b': 'bachelor of arts',
    r'\bm[\s\.-]*a\b': 'master of arts',
    r'\bb[\s\.-]*com\b': 'bachelor of commerce',
    r'\bm[\s\.-]*com\b': 'master of commerce',
    r'\bb[\s\.-]*ba\b': 'bachelor of business administration',
    r'\bm[\s\.-]*ba\b': 'master of business administration',
    r'\bmba\b': 'master of business administration',
    r'\bbba\b': 'bachelor of business administration',
    r'\bph[\s\.-]*d\b': 'doctor of philosophy',
    r'\bd[\s\.-]*phil\b': 'doctor of philosophy',
    }


    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [60]:
df['degree_names_cleaned'] = df['degree_names'].apply(clean_degree_name)

In [13]:
df

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score,degree_names_cleaned
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000,['bachelor of technology']
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000,"['bachelor of science (maths)', 'master of sci..."
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667,['bachelor of technology']
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000,['computer applications specialist certificate...
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000,['bachelor of business administration']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,NaN,NaN,"['Mathematical modelling', 'Machine Learning',...",['Sanghvi College of Engineering'],['B.Tech'],['2019'],['N/A'],['N/A'],['N/A'],['BPM Foundation'],...,NaN,NaN,Data Engineer,Bachelor of Science (BSc),5 to 8 years,NaN,Data Platform Design\nData Pipeline Developmen...,Azure\nBig Data\nData Analytics\nETL Tools\nPo...,0.683333,['bachelor of technology']
9540,NaN,Expertise EDA modeler. I like to learn what my...,"['Data Analysis', 'Business Analysis', 'Machin...","['KVoCT, Pune', 'KVoCT, Pune']","['B.CA', 'M.CA']","['2018', '2020']","[None, None]","[None, None]","[None, None]",['Passionate Solution'],...,NaN,NaN,Executive/ Sr. Executive -IT,Bachelor of Science (BSc) in Computer Science ...,3 to 5 years,Age at most 40 years,Hardware & Software Installation\nSystem Monit...,NaN,0.650000,"['b.ca', 'm.ca']"
9541,NaN,Looking for roles related to application devel...,"['Business Analyst', 'Data Analytics', 'Data C...",['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],['N/A'],['ZigSAW'],...,NaN,NaN,Ex

In [61]:
def clean_educational_requirement(text):
    if pd.isna(text):
        return ''
    
    text = text.lower()

    # Use same abbreviation expansion
    replacements = {
        r'\bb[\s\.-]*sc\b': 'bachelor of science',
        r'\bm[\s\.-]*sc\b': 'master of science',
        r'\bb[\s\.-]*tech\b': 'bachelor of technology',
        r'\bm[\s\.-]*tech\b': 'master of technology',
        r'\bb[\s\.-]*e\b': 'bachelor of engineering',
        r'\bm[\s\.-]*e\b': 'master of engineering',
        r'\bb[\s\.-]*a\b': 'bachelor of arts',
        r'\bm[\s\.-]*a\b': 'master of arts',
        r'\bb[\s\.-]*com\b': 'bachelor of commerce',
        r'\bm[\s\.-]*com\b': 'master of commerce',
        r'\bb[\s\.-]*ba\b': 'bachelor of business administration',
        r'\bm[\s\.-]*ba\b': 'master of business administration',
        r'\bmba\b': 'master of business administration',
        r'\bbba\b': 'bachelor of business administration',
        r'\bph[\s\.-]*d\b': 'doctor of philosophy',
        r'\bd[\s\.-]*phil\b': 'doctor of philosophy',
        # Other useful substitutions
        r'\bbachelor/honors\b': 'bachelor degree',
        r'\bdiploma\b': 'diploma',
        r'\bmasters?\b': 'master degree',  # handles "Masters" or "Master"
    }

    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [62]:
df['educational_requirements_cleaned'] = df['educationaL_requirements'].apply(clean_educational_requirement)

In [64]:
df

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score,degree_names_cleaned,educational_requirements_cleaned
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],[electronics engineering],['Coca-COla'],...,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,1,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000,['bachelor of technology'],bachelor of science in computer science & engi...
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","[mathematics', 'statistics]",['BIB Consultancy'],...,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,5,NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000,"['bachelor of science (maths)', 'master of sci...",master degree of science in computer science &...
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],"[electronics engineering, telecommunication]",['Axis Bank Limited'],...,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),3,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667,['bachelor of technology'],master degree of business administration (mast...
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],"[computer science, applications]","['Company Name ï¼ City , State', 'Company Name...",...,NaN,Business Development Executive,Bachelor/Honors,1,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000,['computer applications specialist certificate...,bachelor degree
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],[accounting],"['Company Name', 'Company Name', 'Company Name...",...,"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,4,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000,['bachelor of business administration'],bachelor of science (bachelor of science) in c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,NaN,NaN,"['Mathematical modelling', 'Machine Learning',...",['Sanghvi College of Engineering'],['B.Tech'],['2019'],['N/A'],['N/A'],[],['BPM Foundation'],...,NaN,Data Engineer,Bachelor of Science (BSc),5,NaN,Data Platform Design\nData Pipeline Developmen...,Azure\nBig Data\nData Analytics\nETL Tools\nPo...,0.683333,['bachelor of technology'],bachelor of science (bachelor of science)
9540,NaN,Expertise EDA modeler. I like to learn what my...,"['Data Analysis', 'Business Analysis', 'Machin...","['KVoCT, Pune', 'KVoCT, Pune']","['B.CA', 'M.CA']","['2018', '2020']","[None, None]","[None, None]",[],['Passionate Solution'],...,NaN,Executive/ Sr. Executive -IT,Bachelor of Science (BSc) in Computer Science ...,3,Age at most 40 years,Hardware & Software Installation\nSystem Monit...,NaN,0.650000,"['b.ca', 'm.ca']",bachelor of science (bachelor of sci

In [63]:
def clean_list_column(entry):
    """
    Converts list or list-like string entries to a space-separated string.
    """
    import ast

    if isinstance(entry, list):
        return ' '.join(entry).lower()
    elif isinstance(entry, str):
        try:
            parsed = ast.literal_eval(entry)
            if isinstance(parsed, list):
                return ' '.join(parsed).lower()
        except:
            pass
        return entry.lower()
    return ''

def combine_degree_and_major(degree, major):
    """
    Combines cleaned degree and major fields into one string for similarity comparison.
    """
    degree_str = clean_list_column(degree)
    major_str = clean_list_column(major)

    if degree_str and major_str:
        return f"{degree_str} in {major_str}"
    elif degree_str:
        return degree_str
    elif major_str:
        return major_str
    else:
        return ''


In [65]:
df['combined_education'] = df.apply(
    lambda row: combine_degree_and_major(row['degree_names_cleaned'], row['major_field_of_studies']),axis=1)


In [66]:
df['combined_education']

0       bachelor of technology in electronics engineering
1       bachelor of science (maths) master of science ...
2       bachelor of technology in electronics engineer...
3       computer applications specialist certificate p...
4       bachelor of business administration in accounting
                              ...                        
9539                               bachelor of technology
9540                                            b.ca m.ca
9541                                   b.bachelor of arts
9542     bachelor of technology in electrical engineering
9543    bachelor of arts in organizational communicati...
Name: combined_education, Length: 9544, dtype: object

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Example: Use your actual DataFrame here
# df['combined_education'] already prepared
# df['educational_requirement_cleaned'] also cleaned (lowercase, no punctuation, etc.)

def compute_cosine_similarity(row, vectorizer):
    texts = [row['combined_education'], row['educational_requirements_cleaned']]
    tfidf = vectorizer.fit_transform(texts)
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
    return similarity

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Compute cosine similarity for each row
df['education_similarity'] = df.apply(
    lambda row: compute_cosine_similarity(row, tfidf_vectorizer), axis=1
)


In [68]:
df

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score,degree_names_cleaned,educational_requirements_cleaned,combined_education,education_similarity
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],[electronics engineering],['Coca-COla'],...,B.Sc in Computer Science & Engineering from a ...,1,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000,['bachelor of technology'],bachelor of science in computer science & engi...,bachelor of technology in electronics engineering,0.318784
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","[mathematics', 'statistics]",['BIB Consultancy'],...,M.Sc in Computer Science & Engineering or in a...,5,NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000,"['bachelor of science (maths)', 'master of sci...",master degree of science in computer science &...,bachelor of science (maths) master of science ...,0.375598
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],"[electronics engineering, telecommunication]",['Axis Bank Limited'],...,Master of Business Administration (MBA),3,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667,['bachelor of technology'],master degree of business administration (mast...,bachelor of technology in electronics engineer...,0.093495
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],"[computer science, applications]","['Company Name ï¼ City , State', 'Company Name...",...,Bachelor/Honors,1,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000,['computer applications specialist certificate...,bachelor degree,computer applications specialist certificate p...,0.000000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],[accounting],"['Company Name', 'Company Name', 'Company Name...",...,Bachelor of Science (BSc) in Computer Science,4,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000,['bachelor of business administration'],bachelor of science (bachelor of science) in c...,bachelor of business administration in accounting,0.312103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,NaN,NaN,"['Mathematical modelling', 'Machine Learning',...",['Sanghvi College of Engineering'],['B.Tech'],['2019'],['N/A'],['N/A'],[],['BPM Foundation'],...,Bachelor of Science (BSc),5,NaN,Data Platform Design\nData Pipeline Developmen...,Azure\nBig Data\nData Analytics\nETL Tools\nPo...,0.683333,['bachelor of technology'],bachelor of science (bachelor of science),bachelor of technology,0.503103
9540,NaN,Expertise EDA modeler. I like to learn what my...,"['Data Analysis', 'Business Analysis', 'Machin...","['KVoCT, Pune', 'KVoCT, Pune']","['B.CA', 'M.CA']","['2018', '2020']","[None, None]","[None, None]",[],['Passionate Solution'],...,Bachelor of Science (BSc) in Computer Science ...,3,Age at most 40 years,Hardware & Software Installation\